In [1]:
import pandas as pd
import numpy as np
import math
import os # needed for addressing files by path when reading and writing
from scipy.io import arff #needed to read headers
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from torch.optim.lr_scheduler import StepLR
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import DataLoader, TensorDataset


/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


import pandas as pd
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azureml.core import Workspace, Experiment, Run

ws = Workspace.from_config()
ml_client = MLClient.from_config(credential=DefaultAzureCredential())
data_asset = ml_client.data.get("naticus", version="1")

# Create an experiment
experiment_name = 'naticus'
experiment = Experiment(ws, experiment_name)

# Start a run
run = experiment.start_logging()
Alldata = pd.read_csv(data_asset.path)
Alldata

Found the config file in: /config.json


,android.permission.GET_ACCOUNTS,com.sonyericsson.home.permission.BROADCAST_BADGE,android.permission.READ_PROFILE,android.permission.MANAGE_ACCOUNTS,android.permission.WRITE_SYNC_SETTINGS,android.permission.READ_EXTERNAL_STORAGE,android.permission.RECEIVE_SMS,com.android.launcher.permission.READ_SETTINGS,android.permission.WRITE_SETTINGS,com.google.android.providers.gsf.permission.READ_GSERVICES,...,com.android.launcher.permission.UNINSTALL_SHORTCUT,com.sec.android.iap.permission.BILLING,com.htc.launcher.permission.UPDATE_SHORTCUT,com.sec.android.provider.badge.permission.WRITE,android.permission.ACCESS_NETWORK_STATE,com.google.android.finsky.permission.BIND_GET_INSTALL_REFERRER_SERVICE,com.huawei.android.launcher.permission.READ_SETTINGS,android.permission.READ_SMS,android.permission.PROCESS_INCOMING_CALLS,Result
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29327,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
29328,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
29329,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
29330,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [3]:
train_split=int(0.8*len(Alldata))
x_train=Alldata[:train_split]
x_test=Alldata[train_split:]
x_test_cpy=x_test
y_train=x_train['Result']
y_test=x_test['Result']

In [4]:
x_train=x_train.drop(columns='Result')
x_test=x_test.drop(columns='Result')
x_test

,android.permission.GET_ACCOUNTS,com.sonyericsson.home.permission.BROADCAST_BADGE,android.permission.READ_PROFILE,android.permission.MANAGE_ACCOUNTS,android.permission.WRITE_SYNC_SETTINGS,android.permission.READ_EXTERNAL_STORAGE,android.permission.RECEIVE_SMS,com.android.launcher.permission.READ_SETTINGS,android.permission.WRITE_SETTINGS,com.google.android.providers.gsf.permission.READ_GSERVICES,...,android.permission.CLEAR_APP_CACHE,com.android.launcher.permission.UNINSTALL_SHORTCUT,com.sec.android.iap.permission.BILLING,com.htc.launcher.permission.UPDATE_SHORTCUT,com.sec.android.provider.badge.permission.WRITE,android.permission.ACCESS_NETWORK_STATE,com.google.android.finsky.permission.BIND_GET_INSTALL_REFERRER_SERVICE,com.huawei.android.launcher.permission.READ_SETTINGS,android.permission.READ_SMS,android.permission.PROCESS_INCOMING_CALLS
23465,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
23466,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
23467,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
23468,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
23469,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29327,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
29328,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
29329,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
29330,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# Standardize the features
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [6]:
import torch.nn as nn
import torch.nn.functional as F

class MalwareDetectionNN(nn.Module):
    def __init__(self, input_size):
        super(MalwareDetectionNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.dropout1 = nn.Dropout(0.5)

        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.dropout2 = nn.Dropout(0.5)

        self.fc3 = nn.Linear(64, 32)  
        self.bn3 = nn.BatchNorm1d(32)
        self.dropout3 = nn.Dropout(0.5)


        self.fc4 = nn.Linear(32, 2)# 2 output classes: benign or malicious

    def forward(self, x):
        x = self.dropout1(F.relu(self.bn1(self.fc1(x))))
        x = self.dropout2(F.relu(self.bn2(self.fc2(x))))
        x = self.dropout3(F.relu(self.bn3(self.fc3(x))))

        x = self.fc4(x)
        #x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        #x = F.relu(self.fc3(x))
      
        return x

In [7]:
all_predictions = []


In [8]:
# Initialize model, loss function, and optimizer
input_size = x_train.shape[1]
model = MalwareDetectionNN(input_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

# Training loop
epochs = 40
best_val_loss = float('inf')
early_stop_patience = 5
no_improve_epochs = 0



for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    scheduler.step()
    
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')
    

In [9]:
torch.save(model.state_dict(), 'malware_detection_model2.pth')

# Load the model
#model = MalwareDetectionNN(input_size)
model.load_state_dict(torch.load('malware_detection_model2.pth'))

<All keys matched successfully>

# Evaluate the model
val_loss = 0.0
correct = 0
total = 0
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_predictions.append(predicted.cpu())  # Append predictions to the list
avg_train_loss = running_loss / len(train_loader)
avg_val_loss = val_loss / len(test_loader)
val_accuracy = 100 * correct / total
print(f'Accuracy: {100 * correct / total}%')

 # Early stopping
if avg_val_loss < best_val_loss:
    best_val_loss = avg_val_loss
    no_improve_epochs = 0
    torch.save(model.state_dict(), 'best_model.pth')  # Save the best model
else:
    no_improve_epochs += 1

if no_improve_epochs >= early_stop_patience:
    print("Early stopping triggered")
    break

In [10]:
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    # Update the learning rate
    scheduler.step()
    
    # Validation step
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    avg_train_loss = running_loss / len(train_loader)
    avg_val_loss = val_loss / len(test_loader)
    val_accuracy = 100 * correct / total
    
    print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}, Val Accuracy: {val_accuracy}%')
    
    # Early stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        no_improve_epochs = 0
        torch.save(model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        no_improve_epochs += 1
    
    if no_improve_epochs >= early_stop_patience:
        print("Early stopping triggered")
        break

Epoch 1, Train Loss: 0.5449257616451066, Val Loss: 0.6280819592673493, Val Accuracy: 70.17214930969831%
Epoch 2, Train Loss: 0.3051504752616467, Val Loss: 0.44772168374417914, Val Accuracy: 78.7966592807227%
Epoch 3, Train Loss: 0.21866272793884822, Val Loss: 0.48146793501370627, Val Accuracy: 77.39901141980569%
Epoch 4, Train Loss: 0.18533483012617447, Val Loss: 0.531015581400741, Val Accuracy: 74.24578148968808%
Epoch 5, Train Loss: 0.1735201141307237, Val Loss: 0.5267753245450718, Val Accuracy: 76.20589739219362%
Epoch 6, Train Loss: 0.16540618924733647, Val Loss: 0.42644892974614934, Val Accuracy: 82.52940173853759%
Epoch 7, Train Loss: 0.15823949453464078, Val Loss: 0.6771737047592583, Val Accuracy: 69.79717061530594%
Epoch 8, Train Loss: 0.1514897797718563, Val Loss: 0.551412502189835, Val Accuracy: 74.67189364240669%
Epoch 9, Train Loss: 0.14724581409499496, Val Loss: 0.6643689273193519, Val Accuracy: 70.06988239304584%
Epoch 10, Train Loss: 0.14450427317326986, Val Loss: 0.6688

In [11]:
import joblib
from azureml.core import Model

os.makedirs('outputs', exist_ok=True)
joblib.dump(model, 'outputs/naticus.pkl')


['outputs/naticus.pkl']

In [12]:
# Save the final model state
model_save_path = 'final_modelNaticus.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Saved final model state to {model_save_path}")

# Register the model
#model = run.register_model(model_name='your_model_name', model_path='outputs/model.pkl')
model = Model.register(
    workspace=ws,
    model_path=model_save_path,
    model_name='Naticus',
    description='A Naticus neural network for malware detection'
)
# Complete the run
run.complete()

Saved final model state to final_modelNaticus.pth
Registering model Naticus
